In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import numpy as np
from math import sqrt, pi, log, e
from enum import Enum
import scipy.stats as stat
from scipy.stats import norm
from math import exp, sqrt

import time



In [ ]:
class BSMerton(object):
    # ... (other methods of BSMerton class) ...

    def approximate_forward_difference(self, func, h=0.01):
        fx = func()
        fx_plus_h = func(h)
        return (fx_plus_h - fx) / h

    def approximate_backward_difference(self, func, h=0.01):
        fx = func()
        fx_minus_h = func(-h)
        return (fx - fx_minus_h) / h

    # Forward and backward difference approximations for Vega
    def vega_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.vega, h)

    def vega_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.vega, h)

    # Forward and backward difference approximations for Theta
    def theta_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.theta, h)

    def theta_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.theta, h)

    # Forward and backward difference approximations for Gamma
    def gamma_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.gamma, h)

    def gamma_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.gamma, h)


class BSMerton:
    # ... (previous code) ...

    def approximate_delta_fd(self, h=0.01):
        # Save the original underlying asset price
        original_S = self.S

        # Set the underlying asset price to the given value
        self.S += h

        # Calculate d1 and d2 with the new underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price_plus_h = self.premium()[0]

        # Reset the underlying asset price to its original value
        self.S = original_S

        # Recalculate d1 and d2 with the original underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price = self.premium()[0]

        # Calculate the approximate delta using forward differencing
        approximate_delta = (option_price_plus_h - option_price) / h

        return approximate_delta




In [69]:
class BSMerton(object):
    def __init__(self, args,b=None):
        self.Type = int(args[0])  # 1 for a Call, - 1 for a put
        self.S = float(args[1])  # Underlying asset price
        self.K = float(args[2])  # Option strike K
        self.r = float(args[3])  # Continuous risk fee rate
        self.q = float(args[4])  # Dividend continuous rate
        self.T = float(args[5]) / 365.0  # Compute time to expiry
        self.sigma = float(args[6])  # Underlying volatility
        self.sigmaT = self.sigma * self.T ** 0.5  # sigma*T for reusability
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT
        if b is None:
            self.b = self.r - self.q
        else:
            self.b = b

        [self.Premium] = self.premium()
        [self.Delta] = self.delta()
        [self.Theta] = self.theta()
        [self.Rho] = self.rho()
        [self.CarryRho] = self.carry_rho()
        [self.Vega] = self.vega()
        [self.Gamma] = self.gamma()
        [self.Phi] = self.phi()
        [self.Charm] = self.dDeltadTime()
        [self.Vanna] = self.dDeltadVol()

    def premium(self):
        tmpprem = self.Type * (self.S * e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1) - \
                               self.K * e ** (-self.r * self.T) * norm.cdf(self.Type * self.d2))
        return [tmpprem]

    ############################################
    ############ 1st order greeks ##############
    ############################################

    def delta(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [dfq * norm.cdf(self.d1)]
        else:
            return [dfq * (norm.cdf(self.d1) - 1)]


    ########THESE ARE ORIGINAL FUNCTIONS FUNCTIONS IN USE ARE TO TRY AND IMPLEMENT FORWARD AND BACKWARD DIFFS#######
    #############################################################################################################
    # # Vega for 1% change in vol
    # def vega(self):
    #     return [0.01 * self.S * e ** (-self.q * self.T) * \
    #             norm.pdf(self.d1) * self.T ** 0.5]

    # # Theta for 1 day change
    # def theta(self):
    #     df = e ** -(self.r * self.T)
    #     dfq = e ** (-self.q * self.T)
    #     tmptheta = (1.0 / 365.0) \
    #                * (-0.5 * self.S * dfq * norm.pdf(self.d1) * \
    #                   self.sigma / (self.T ** 0.5) + \
    #                   self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) \
    #                                - self.r * self.K * df * norm.cdf(self.Type * self.d2)))
    #     return [tmptheta]

    def vega(self, h=0.0):
        return [0.01 * self.S * e ** (-self.q * (self.T - h)) * \
            norm.pdf(self.d1) * (self.T - h) ** 0.5]

    def theta(self, h=0.0):
        df = e ** -(self.r * (self.T - h))
        dfq = e ** (-self.q * (self.T - h))
        tmptheta = (1.0 / 365.0) \
                * (-0.5 * self.S * dfq * norm.pdf(self.d1) * \
                    self.sigma / ((self.T - h) ** 0.5) + \
                    self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) \
                                - self.r * self.K * df * norm.cdf(self.Type * self.d2)))
        return [tmptheta]

    def gamma(self, h=0.0):
        return [e ** (-self.q * (self.T - h)) * norm.pdf(self.d1) / (self.S * self.sigmaT)]


    def rho(self):
        df = e ** -(self.r * self.T)
        return [self.Type * self.K * self.T * df * 0.01 * norm.cdf(self.Type * self.d2)]
    
    from numpy import e

    def carry_rho(self):
        T = self.T
        S = self.S
        r = self.r
        q = self.q
        b = self.b
        d1 = self.d1
        d2 = self.d2
        df = e ** (-r * T)
        dfb = e ** ((b - r) * T)

        if self.Type == 1:  # Call
            carry_rho = T * S * dfb * norm.cdf(d1)
        else:  # Put
            carry_rho = -T * S * dfb * norm.cdf(-d1)

        return [carry_rho * 0.01]


    def phi(self):
        return [0.01 * -self.Type * self.T * self.S * \
                e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1)]

    ############################################
    ############ 2nd order greeks ##############
    ############################################

    ########THESE ARE ORIGINAL FUNCTIONS FUNCTIONS IN USE ARE TO TRY AND IMPLEMENT FORWARD AND BACKWARD DIFFS#######
    #############################################################################################################
    # def gamma(self):
    #     return [e ** (-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigmaT)]

    # Charm for 1 day change
    def dDeltadTime(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + (-self.q) * norm.cdf(self.d1))]
        else:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + self.q * norm.cdf(-self.d1))]

    # Vanna for 1% change in vol
    def dDeltadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

    # Vomma
    def dVegadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]
    
    def approximate_forward_difference(self, func, h):
        fx = func()[0]

        # Modify the required variables
        if func == self.vega:
            self.sigma += h
        elif func == self.theta:
            self.T += h
        elif func == self.gamma:
            self.S += h

        # Recalculate the function value with the modified variables
        fx_plus_h = func()[0]

        # Restore the original variables
        if func == self.vega:
            self.sigma -= h
        elif func == self.theta:
            self.T -= h
        elif func == self.gamma:
            self.S -= h

        return (fx_plus_h - fx) / h

    def approximate_backward_difference(self, func, h):
        fx = func()[0]

        # Modify the required variables
        if func == self.vega:
            self.sigma -= h
        elif func == self.theta:
            self.T -= h
        elif func == self.gamma:
            self.S -= h

        # Recalculate the function value with the modified variables
        fx_minus_h = func()[0]

        # Restore the original variables
        if func == self.vega:
            self.sigma += h
        elif func == self.theta:
            self.T += h
        elif func == self.gamma:
            self.S += h

        return (fx - fx_minus_h) / h



    # Forward and backward difference approximations for Vega
    def vega_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.vega, h)

    def vega_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.vega, h)

    # Forward and backward difference approximations for Theta
    def theta_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.theta, h)

    def theta_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.theta, h)

    # Forward and backward difference approximations for Gamma
    def gamma_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.gamma, h)

    def gamma_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.gamma, h)
    
    def approximate_delta_fd(self, h=0.01):
        # Save the original underlying asset price
        original_S = self.S

        # Set the underlying asset price to the given value
        self.S += h

        # Calculate d1 and d2 with the new underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price_plus_h = self.premium()[0]

        # Reset the underlying asset price to its original value
        self.S = original_S

        # Recalculate d1 and d2 with the original underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price = self.premium()[0]

        # Calculate the approximate delta using forward differencing
        approximate_delta = (option_price_plus_h - option_price) / h

        return approximate_delta
    def approximate_delta_bd(self, h=0.01):
        # Save the original underlying asset price
        original_S = self.S

        # Set the underlying asset price to the given value
        self.S -= h

        # Calculate d1 and d2 with the new underlying asset price
        self.d1 = (log(self.S / self.K) + \
                (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price_minus_h = self.premium()[0]

        # Reset the underlying asset price to its original value
        self.S = original_S

        # Recalculate d1 and d2 with the original underlying asset price
        self.d1 = (log(self.S / self.K) + \
                (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price = self.premium()[0]

        # Calculate the approximate delta using backward differencing
        approximate_delta = (option_price - option_price_minus_h) / h

        return approximate_delta



In [18]:
import datetime as dt
from datetime import date
expiration = dt.datetime(2022,4,15)
current_date = dt.datetime(2022,3,13)

delta= expiration - current_date
delta.days

33

In [67]:
AAPLCall = [1, 151.03, 165, 0.0425, 0.0053, 33, 0.20]
bsm = BSMerton(AAPLCall)
vega_fd_approx = bsm.gamma_bd_approx(h=2)
print(vega_fd_approx)

# vega_bd_approx = bsm.vega_bd_approx(h=1)
# print(vega_bd_approx)


-0.00011288274912334834


In [66]:
AAPLCall = [1, 151.03, 165, 0.0425, 0.0053, 33, 0.20]
bsm = BSMerton(AAPLCall)
vega_fd_approx = bsm.vega_fd_approx(h=35)
vega_bd_approx = bsm.vega_bd_approx(h=35)
print(vega_fd_approx)
print(vega_bd_approx)


0.0
0.0


In [128]:
# For general accuracy purposes I looked up Implied volatility for AAPL back on 3/13/2022, it was 30.37%


AAPLCall = [1,151.03,165,.0425,.0053, 33,.20]
AAPLPut = [-1,151.03,165,.0425,.0053, 33,.2]

AAPL_Call_Premium = BSMerton(AAPLCall).Premium
AAPL_Call_Delta = BSMerton(AAPLCall).Delta
AAPL_Call_Gamma = BSMerton(AAPLCall).Gamma
AAPL_Call_Theta = BSMerton(AAPLCall).Theta
AAPL_Call_Vega = BSMerton(AAPLCall).Vega
AAPL_Call_Rho = BSMerton(AAPLCall).Rho
AAPL_Call_Rho_Carry = BSMerton(AAPLCall).CarryRho
AAPL_Call_Charm = BSMerton(AAPLCall).Charm
AAPL_Call_Vanna = BSMerton(AAPLCall).Vanna


AAPL_Call_Premium_Value = "AAPL_Call_Premium_Value"
AAPL_Call_Delta_Exposure = "AAPL Net Delta Exposure: "
AAPL_Call_Gamma_Exposure = "AAPL Net Gamma Exposure: "
AAPL_Call_Theta_Exposure = "AAPL Net Theta Exposure: "
AAPL_Call_Vega_Exposure = "AAPL Net Vega Exposure: "
AAPL_Call_Rho_Exposure = "AAPL Net Rho Exposure: "
AAPL_Call_Rho_Carry_Exposure = "AAPL Net Carry Rho Exposure: "
AAPL_Call_Charm_Exposure = "AAPL Net Charm Exposure: "
AAPL_Call_Vanna_Exposure = "AAPL Net Vanna Exposure: "

print(AAPL_Call_Premium_Value,AAPL_Call_Premium)
print(AAPL_Call_Delta_Exposure,AAPL_Call_Delta)
print(AAPL_Call_Gamma_Exposure,AAPL_Call_Gamma)
print(AAPL_Call_Theta_Exposure,AAPL_Call_Theta)
print(AAPL_Call_Vega_Exposure,AAPL_Call_Vega)
print(AAPL_Call_Rho_Exposure,AAPL_Call_Rho)
print(AAPL_Call_Rho_Carry_Exposure,AAPL_Call_Rho_Carry)
print(AAPL_Call_Charm_Exposure,AAPL_Call_Charm)
print(AAPL_Call_Vanna_Exposure,AAPL_Call_Vanna)


AAPL_Call_Premium_Value 0.3357989976315192
AAPL Net Delta Exposure:  0.08297130333914773
AAPL Net Gamma Exposure:  0.016822916101852648
AAPL Net Theta Exposure:  -0.022264444821010514
AAPL Net Vega Exposure:  0.06938710929513443
AAPL Net Rho Exposure:  0.01102593915636819
AAPL Net Carry Rho Exposure:  0.01132953825011723
AAPL Net Charm Exposure:  -0.003603543622813535
AAPL Net Vanna Exposure:  0.011041137391941661


In [29]:

AAPL_Put_Delta = BSMerton(AAPLPut).Delta
AAPL_Put_Gamma = BSMerton(AAPLPut).Gamma
AAPL_Put_Theta = BSMerton(AAPLPut).Theta
AAPL_Put_Vega = BSMerton(AAPLPut).Vega
AAPL_Put_Rho = BSMerton(AAPLPut).Rho
AAPL_Put_Rho_Carry = BSMerton(AAPLPut).CarryRho
AAPL_Put_Charm = BSMerton(AAPLPut).Charm
AAPL_Put_Vanna = BSMerton(AAPLPut).Vanna

AAPL_Put_Delta_Exposure = "AAPL Net Delta Exposure: "
AAPL_Put_Gamma_Exposure = "AAPL Net Gamma Exposure: "
AAPL_Put_Theta_Exposure = "AAPL Net Theta Exposure: "
AAPL_Put_Vega_Exposure = "AAPL Net Vega Exposure: "
AAPL_Put_Rho_Exposure = "AAPL Net Rho Exposure: "
AAPL_Put_Rho_Carry_Exposure = "AAPL Net Carry Rho Exposure: "
AAPL_Put_Charm_Exposure = "AAPL Net Charm Exposure: "
AAPL_Put_Vanna_Exposure = "AAPL Net Vanna Exposure: "

print(AAPL_Put_Delta_Exposure,AAPL_Put_Delta)
print(AAPL_Put_Gamma_Exposure,AAPL_Put_Gamma)
print(AAPL_Put_Theta_Exposure,AAPL_Put_Theta)
print(AAPL_Put_Vega_Exposure,AAPL_Put_Vega)
print(AAPL_Put_Rho_Exposure,AAPL_Put_Rho)
print(AAPL_Put_Rho_Carry_Exposure,AAPL_Put_Rho_Carry)
print(AAPL_Put_Charm_Exposure,AAPL_Put_Charm)
print(AAPL_Put_Vanna_Exposure,AAPL_Put_Vanna)


AAPL Net Delta Exposure:  -0.9165496333661425
AAPL Net Gamma Exposure:  0.016822916101852648
AAPL Net Theta Exposure:  -0.005317784872060155
AAPL Net Vega Exposure:  0.06938710929513443
AAPL Net Rho Exposure:  -0.1375800312273579
AAPL Net Carry Rho Exposure:  -0.1251527180054937
AAPL Net Charm Exposure:  -0.0036180572144972013
AAPL Net Vanna Exposure:  0.011041137391941661


In [140]:
from math import exp, sqrt
import numpy as np

def bt_american(call, underlying, strike, ttm, rf, b, ivol, N):
    dt = ttm / N
    u = exp(ivol * sqrt(dt))
    d = 1 / u
    pu = (exp(b * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = exp(-rf * dt)
    z = 1 if call else -1

    def nNodeFunc(n):
        return int((n + 1) * (n + 2) // 2)

    def idxFunc(i, j):
        return nNodeFunc(j - 1) + i

    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes)

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = underlying * u ** i * d ** (j - i)
            optionValues[idx] = max(0, z * (price - strike))

            if j < N:
                optionValues[idx] = max(optionValues[idx], df * (pu * optionValues[idxFunc(i + 1, j + 1)] + pd * optionValues[idxFunc(i, j + 1)]))

    return optionValues[0]


def binomial_tree_with_div(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N):
    if not divAmts or not divTimes or divTimes[0] > N:
        return bt_american(call, underlying, strike, ttm, rf, rf, ivol, N)

    dt = ttm / N
    u = exp(ivol * sqrt(dt))
    d = 1 / u
    pu = (exp(rf * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = exp(-rf * dt)
    z = 1 if call else -1

    def nNodeFunc(n):
        return int((n + 1) * (n + 2) // 2)

    def idxFunc(i, j):
        return nNodeFunc(j - 1) + i

    nDiv = len(divTimes)
    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes)

    for j in range(int(divTimes[0]), -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = underlying * u ** i * d ** (j - i)

            if j < divTimes[0]:
                optionValues[idx] = max(0, z * (price - strike))
                optionValues[idx] = max(optionValues[idx], df * (pu * optionValues[idxFunc(i + 1, j + 1)] + pd * optionValues[idxFunc(i, j + 1)]))
            else:
                valNoExercise = binomial_tree_with_div(call, price - divAmts[0], strike, ttm - divTimes[0] * dt, rf, divAmts[1:], [t - divTimes[0] for t in divTimes[1:]], ivol, N - divTimes[0])
                valExercise = max(0, z * (price - strike))
                optionValues[idx] = max(valNoExercise, valExercise)

    return optionValues[0]

call = True
underlying = 151.03
strike = 165
ttm = 33/365
rf = 0.0425
b = 0.0425
divAmts =  [0.88]
divTimes = [29/365]
ivol = 0.20
N = 300

# AAPL_option_call = binomial_tree_with_div(True, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
AAPL_option_call_no_div = bt_american(call, underlying, strike, ttm, rf, b, ivol, N)
print(f"Option price: {AAPL_option_call_no_div:.2f}")


Option price: 0.34


In [142]:
def calculate_greeks(option_func, call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N):
    dS = 0.01 * underlying
    dV = 0.01 * ivol

    if option_func == bt_american:
        P = option_func(call, underlying, strike, ttm, rf, rf, ivol, N)
        P_up = option_func(call, underlying + dS, strike, ttm, rf, rf, ivol, N)
        P_down = option_func(call, underlying - dS, strike, ttm, rf, rf, ivol, N)
        P_vol_up = option_func(call, underlying, strike, ttm, rf, rf, ivol + dV, N)
        P_vol_down = option_func(call, underlying, strike, ttm, rf, rf, ivol - dV, N)
    else:
        P = option_func(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
        P_up = option_func(call, underlying + dS, strike, ttm, rf, divAmts, divTimes, ivol, N)
        P_down = option_func(call, underlying - dS, strike, ttm, rf, divAmts, divTimes, ivol, N)
        P_vol_up = option_func(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol + dV, N)
        P_vol_down = option_func(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol - dV, N)

    delta = (P_up - P_down) / (2 * dS)
    gamma = (P_up - 2 * P + P_down) / (dS ** 2)
    vega = (P_vol_up - P_vol_down) / (2 * dV)

    return delta, gamma, vega


# For binomial_tree_with_div
delta, gamma, vega = calculate_greeks(binomial_tree_with_div, call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
print(f"Delta: {delta:.4f}\nGamma: {gamma:.4f}\nVega: {vega:.4f}")

print("\n")

# For bt_american
delta, gamma, vega = calculate_greeks(bt_american, call, underlying, strike, ttm, rf, [], [], ivol, N)
print(f"Delta: {delta:.4f}\nGamma: {gamma:.4f}\nVega: {vega:.4f}")

Delta: 0.0000
Gamma: 0.0000
Vega: -0.0052


Delta: 0.0854
Gamma: 0.0170
Vega: 7.3056
